In [1]:
import pandas as pd
from bertopic import BERTopic
from sklearn.decomposition import PCA

import warnings 
warnings.filterwarnings('ignore')

/Users/andrejvaluhov/Documents/test_tasks/svcb_test/.scb_test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Topic Modeling

In [2]:
pd.set_option("max_colwidth", 100)

Загрузим и преобразуем данные также, как делали в первом задании

In [3]:
data = pd.read_csv('data/topic_modeling_task_sample_trainPart.csv')
data['text_employer'] = data['text_employer'].apply(lambda x: x.replace('_', ' ').lower().strip())
data['text_employer'] = data['text_employer'].str.split('.')

# посчитаем количество фраз в диалоге - в задании от нас требуется привести порядковый номер фразы
data['ordinal'] = data['text_employer'].apply(lambda x: list(range(len(x))))
data = data.explode(['text_employer', 'ordinal'])

In [4]:
data

,Unnamed: 0.1,Unnamed: 0,ucid,text_employer,date,ACTION_ITEM_RESULT_PRODUCT_NAME,ordinal
0,3,7,"5,01000641613474E+017",добрый день,2021-02-16,Бизнес-карта,0
0,3,7,"5,01000641613474E+017",это клиентский менеджер виктория сбер бизнес звоню вам как индивидуальный предприниматель вам у...,2021-02-16,Бизнес-карта,1
0,3,7,"5,01000641613474E+017",я не знаем,2021-02-16,Бизнес-карта,2
0,3,7,"5,01000641613474E+017",я не пойму много вашего времени предложение которую я хотела с вами обсудить я звоню рассказать...,2021-02-16,Бизнес-карта,3
0,3,7,"5,01000641613474E+017",и возможности подключить бизнес кэшбэк чтобы зарабатывать на своих покупках если вы заберете ка...,2021-02-16,Бизнес-карта,4
...,...,...,...,...,...,...,...
42764,138905,350816,"5,01036981613722E+017",ориентировочно в это же время,2021-02-19,Бизнес-карта,9
42764,138905,350816,"5,01036981613722E+017",будет удобно,2021-02-19,Бизнес-карта,10
42764,138905,350816,"5,01036981613722E+017",какие-то вопросы может быть по другим продукт банка на сегодня день у вас есть,2021-02-19,Бизнес-карта,11
42764,138905,350816,"5,01036981613722E+017",поняла вас все тогда вам перезвоню следующую среду ориентировочно в это же время благодарю вас ...,2021-02-19,Бизнес-карта,12


Всего во всех диалогах получилось почти полмиллиона фраз. Для демонстрационных целей ограничимся первыми 20 000 фраз

In [5]:
phrases = data['text_employer'][:20000].copy()

Делать моделирование будем на уникальных фразах.

In [6]:
unique_phrases = phrases.unique()
len(unique_phrases)

17183

In [7]:
# сделаем словари сопоставления уникальной фразы и ее id, и наоборот
phrase2id = {k: v for k, v in enumerate(unique_phrases)}
id2phrase = {v: k for k, v in phrase2id.items()}

Topic Modeling будем делать при помощи библиотеки BERTopic

Осуществим направленное моделирование - создадим seed_list с потенциальными топиками.

In [8]:
seed_topic_list = [ 
                    ['приветствие'],
                    ['перенос активности'],
                    ['окончание разговора'],
                    ['прочее'],
                    ['следующие шаги'],
                    ['безопасность'],
                    ['тарифы и цены'],
                    ['акции'],
                    ['удобство работы'],
                    ['простота настройки'],
                    ['скорость работы'],
                    ['преимущества'],
                    ['вовлечение (подведение к диалогу)'],
                    ['призыв к действию'],
                    ['инструкция по подключению/приобретению'],
                    ['вопрос клиента']
]

In [9]:
# для снижения размерности воспользуемся PCA - будем сжимать 192 размерностей вектора в 300
dim_model = PCA(n_components=300)

In [10]:
# В качестве энкодера реплик возьмем cointegrated/LaBSE-en-ru
# https://huggingface.co/cointegrated/LaBSE-en-ru
topic_model = BERTopic(
    embedding_model="cointegrated/LaBSE-en-ru", 
    umap_model=dim_model,
    verbose=True,
    seed_topic_list=seed_topic_list)

Для кластеризации будем использовать HDBSCAN

In [11]:
topic_model.hdbscan_model

HDBSCAN(min_cluster_size=10, prediction_data=True)

In [12]:
%%time
topics, probs = topic_model.fit_transform(unique_phrases)

2024-05-31 10:13:17,817 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|████████████████████████████████| 537/537 [06:28<00:00,  1.38it/s]
2024-05-31 10:19:50,894 - BERTopic - Embedding - Completed ✓
2024-05-31 10:19:50,895 - BERTopic - Guided - Find embeddings highly related to seeded topics.
Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  6.47it/s]
2024-05-31 10:19:51,256 - BERTopic - Guided - Completed ✓
2024-05-31 10:19:51,257 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-31 10:19:51,462 - BERTopic - Dimensionality - Completed ✓
2024-05-31 10:19:51,513 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-31 10:24:37,095 - BERTopic - Cluster - Completed ✓
2024-05-31 10:24:37,103 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-31 10:24:37,368 - BERTopic - Representation - Completed ✓


CPU times: user 29min 33s, sys: 53 s, total: 30min 26s
Wall time: 11min 19s


Ниже можно посмотреть на все построенные топики, большинство - это "мусорный кластер" с меткой -1

In [13]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,14609,-1_вас_по_вы_на,"[вас, по, вы, на, вам, не, то, будет, есть, если]",[ звоню так как вам доступно специальное предложение по бизнес-карте если вы выпустить я оформит...
1,0,1012,0_менеджер_звоню_удобно_индивидуальный,"[менеджер, звоню, удобно, индивидуальный, предприниматель, персональный, говорить, сбер, сбербан...",[ это ваш персональный менеджер сбербанк блок сбер бизнес я звоню вам как индивидуальный предпри...
2,1,463,1_свидания_доброго_всего_уделенное,"[свидания, доброго, всего, уделенное, хорошего, спасибо, до, дня, время, за]","[ спасибо вам за уделенное время всего доброго хорошего дня до свидания, спасибо вам за уделенн..."
3,2,243,2_эта_преимущества_круглосуточно_покупок,"[эта, преимущества, круглосуточно, покупок, карта, для, привязана, основной, снимать, предложение]",[ звоню рассказать что для вас доступно специальное предложение по бизнес-карте эта карта позвол...
4,3,85,3_дистанционно_большинство_свои_помочь,"[дистанционно, большинство, свои, помочь, буду, номера, рада, контакты, направлю, звоните]",[ хорошо тогда направлю вам свои контакты если потребуется оформить услуги со сбербанка звоните ...
5,4,69,4_что_вот_во_так,"[что, вот, во, так, такая, далее, после, ли, он, которых]","[ что что, что вот, а вот что]"
6,5,62,5_виза_мастер_мастеркард_карт,"[виза, мастер, мастеркард, карт, тип, или, платежная, система, карты, разницы]","[ хорошо скажите тип карты будет мастер карт или виза, а и тип карты мастер карт или виза, хор..."
7,6,62,6_продуктам_вопросы_услуга_банка,"[продуктам, вопросы, услуга, банка, какие, может, быть, ко, возможно, по]","[ все хорошо я вас с ко мне может быть какие-то вопросы по продуктам услуга банка, какие-то воп..."
8,7,62,7_вопросы_какие_остались_ко,"[вопросы, какие, остались, ко, мне, еще, есть, то, может, вопрос]","[ вопросы еще какие-то есть у вас, хорошо ко мне какие-то вопросы может быть сейчас у вас есть,..."
9,8,57,8_удобно_говорить_сейчас_разговаривать,"[удобно, говорить, сейчас, разговаривать, приятно, вам, очень, скажите, будет, поговорить]","[ сейчас вам удобно говорить, сейчас удобно говорить вам, сейчас удобно вам говорить]"


Ниже данные о каждой уникальной фразе - ее топик, макс. вероятность и топ-слов для этого топика.

In [14]:
result = topic_model.get_document_info(unique_phrases)
result

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,добрый день,1,1_свидания_доброго_всего_уделенное,"[свидания, доброго, всего, уделенное, хорошего, спасибо, до, дня, время, за]","[ спасибо вам за уделенное время всего доброго хорошего дня до свидания, спасибо вам за уделенн...",свидания - доброго - всего - уделенное - хорошего - спасибо - до - дня - время - за,1.000000,False
1,это клиентский менеджер виктория сбер бизнес звоню вам как индивидуальный предприниматель вам у...,0,0_менеджер_звоню_удобно_индивидуальный,"[менеджер, звоню, удобно, индивидуальный, предприниматель, персональный, говорить, сбер, сбербан...",[ это ваш персональный менеджер сбербанк блок сбер бизнес я звоню вам как индивидуальный предпри...,менеджер - звоню - удобно - индивидуальный - предприниматель - персональный - говорить - сбер - ...,0.929377,False
2,я не знаем,-1,-1_вас_по_вы_на,"[вас, по, вы, на, вам, не, то, будет, есть, если]",[ звоню так как вам доступно специальное предложение по бизнес-карте если вы выпустить я оформит...,вас - по - вы - на - вам - не - то - будет - есть - если,0.000000,False
3,я не пойму много вашего времени предложение которую я хотела с вами обсудить я звоню рассказать...,2,2_эта_преимущества_круглосуточно_покупок,"[эта, преимущества, круглосуточно, покупок, карта, для, привязана, основной, снимать, предложение]",[ звоню рассказать что для вас доступно специальное предложение по бизнес-карте эта карта позвол...,эта - преимущества - круглосуточно - покупок - карта - для - привязана - основной - снимать - пр...,0.474755,False
4,и возможности подключить бизнес кэшбэк чтобы зарабатывать на своих покупках если вы заберете ка...,-1,-1_вас_по_вы_на,"[вас, по, вы, на, вам, не, то, будет, есть, если]",[ звоню так как вам доступно специальное предложение по бизнес-карте если вы выпустить я оформит...,вас - по - вы - на - вам - не - то - будет - есть - если,0.000000,False
...,...,...,...,...,...,...,...,...
17178,фамилию имя отчество данное указали строчки и далее откроется данные по мужской дата рождения,-1,-1_вас_по_вы_на,"[вас, по, вы, на, вам, не, то, будет, есть, если]",[ звоню так как вам доступно специальное предложение по бизнес-карте если вы выпустить я оформит...,вас - по - вы - на - вам - не - то - будет - есть - если,0.000000,False
17179,вот по паспортные данные там обычно они автоматически появляются там какие не заполнено нужно з...,-1,-1_вас_по_вы_на,"[вас, по, вы, на, вам, не, то, будет, есть, если]",[ звоню так как вам доступно специальное предложение по бизнес-карте если вы выпустить я оформит...,вас - по - вы - на - вам - не - то - будет - есть - если,0.000000,False
17180,а не сможете сейчас,-1,-1_вас_по_вы_на,"[вас, по, вы, на, вам, не, то, будет, есть, если]",[ звоню так как вам доступно специальное предложение по бизнес-карте если вы выпустить я оформит...,вас - по - вы - на - вам - не - то - будет - есть - если,0.000000,False
17181,вот смотрите здесь самое главное держатель карты указываете у вас открылось заполняете обращай ...,-1,-1_вас_по_вы_на,"[вас, по, вы, на, вам, не, то, будет, есть, если]",[ звоню так как вам доступно специальное предложение по бизнес-карте если вы выпустить я оформит...,вас - по - вы - на - вам - не - то - будет - есть - если,0.000000,False


Приведем результаты моделирования к виду - ucid, порядковый номер фразы в диалоге, сама фраза, топик

In [15]:
phrase_topic_dict = result[['Name', 'Document']].set_index('Document').to_dict()['Name']

In [16]:
final_result = data[:20000][['ucid', 'ordinal', 'text_employer']].reset_index(drop=True)
final_result['topic'] = final_result['text_employer'].map(phrase_topic_dict)

In [18]:
final_result.head(5)

,ucid,ordinal,text_employer,topic
0,"5,01000641613474E+017",0,добрый день,1_свидания_доброго_всего_уделенное
1,"5,01000641613474E+017",1,это клиентский менеджер виктория сбер бизнес звоню вам как индивидуальный предприниматель вам у...,0_менеджер_звоню_удобно_индивидуальный
2,"5,01000641613474E+017",2,я не знаем,-1_вас_по_вы_на
3,"5,01000641613474E+017",3,я не пойму много вашего времени предложение которую я хотела с вами обсудить я звоню рассказать...,2_эта_преимущества_круглосуточно_покупок
4,"5,01000641613474E+017",4,и возможности подключить бизнес кэшбэк чтобы зарабатывать на своих покупках если вы заберете ка...,-1_вас_по_вы_на


In [19]:
final_result.to_csv('data/topic_modeling_result.csv', index=False)